Importing all the necessarylibraries from python

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

Import the dataset

In [20]:
data = pd.read_csv('customer_df.csv')

In [21]:
data.head()

,Customer_ID,Unnamed: 1,Document Number,Document Date,Net due date,Amount in doc. curr.,Document type,Local Currency,Posting Date,Payment date,Invoice reference,Document currency,Eff.exchange rate,Clearing Document,Name 1
0,1001141939,41939,1122001110,24/08/2022,24/08/2022,"35,556,562.19",RV,NGN,24/08/2022,31/08/2022,1122001110,NGN,1,6022011197,Nigerian Foundries Limited
1,1001141939,41939,1122001247,09/09/2022,09/09/2022,"-888,914.05",RV,NGN,09/09/2022,21/09/2022,1122001247,NGN,1,6022012262,Nigerian Foundries Limited
2,1001142255,42255,1122000217,07/03/2022,07/03/2022,"4,970,998.55",RV,NGN,07/03/2022,01/04/2022,1122000217,NGN,1,6022003871,CROWN FLOUR MILLS LIMITED - KADUNA
3,106084175,84175,1122000321,31/03/2022,31/03/2022,"169,578.56",RV,NGN,31/03/2022,31/05/2022,1122000321,NGN,1,6022005677,SODUNKE OLAGOKE NIGERIA ENTERPRISE
4,106084413,84413,1122000225,08/03/2022,08/03/2022,"1,241,303.58",RV,NGN,08/03/2022,29/04/2022,1122000225,NGN,1,6022004248,Debrox Contracting Nigeria Limited


Do some data cleaning to make the dataset ready for business model

In [22]:
# Reduce the customer_id number
data['Customer_ID'] = data['Customer_ID'].apply(lambda x:str(x)[-5:])

In [23]:
# Drop rows with missing critical values
data = data.dropna(subset=['Document Date', 'Net due date', 'Customer_ID', 'Payment date','Posting Date'])

# Convert date columns to datetime format
data['Document Date'] = pd.to_datetime(data['Document Date'], dayfirst=True)
data['Net due date'] = pd.to_datetime(data['Net due date'], dayfirst=True)
data['Payment date'] = pd.to_datetime(data['Payment date'], dayfirst=True)
data['Posting Date'] = pd.to_datetime(data['Posting Date'], dayfirst=True)

In [24]:
# Calculate the days taken to pay an invoice
data['Days to Pay'] = (data['Payment date'] - data['Document Date']).dt.days

# Clean and convert 'Invoice Amount' to numeric
data['Invoice Amount'] = data['Amount in doc. curr.'].str.replace(',', '').astype(float)

# Calculate whether the payment was late
data['Late Payment'] = data['Payment date'] > data['Net due date']

# Create the target variable: 1 for late payers, 0 for early payers
late_threshold = 60
data['Is_Late_Payer'] = (data['Days to Pay'] > late_threshold).astype(int)

In [25]:
data.head()

,Customer_ID,Unnamed: 1,Document Number,Document Date,Net due date,Amount in doc. curr.,Document type,Local Currency,Posting Date,Payment date,Invoice reference,Document currency,Eff.exchange rate,Clearing Document,Name 1,Days to Pay,Invoice Amount,Late Payment,Is_Late_Payer
0,41939,41939,1122001110,2022-08-24,2022-08-24,"35,556,562.19",RV,NGN,2022-08-24,2022-08-31,1122001110,NGN,1,6022011197,Nigerian Foundries Limited,7,35556562.19,True,0
1,41939,41939,1122001247,2022-09-09,2022-09-09,"-888,914.05",RV,NGN,2022-09-09,2022-09-21,1122001247,NGN,1,6022012262,Nigerian Foundries Limited,12,-888914.05,True,0
2,42255,42255,1122000217,2022-03-07,2022-03-07,"4,970,998.55",RV,NGN,2022-03-07,2022-04-01,1122000217,NGN,1,6022003871,CROWN FLOUR MILLS LIMITED - KADUNA,25,4970998.55,True,0
3,84175,84175,1122000321,2022-03-31,2022-03-31,"169,578.56",RV,NGN,2022-03-31,2022-05-31,1122000321,NGN,1,6022005677,SODUNKE OLAGOKE NIGERIA ENTERPRISE,61,169578.56,True,1
4,84413,84413,1122000225,2022-03-08,2022-03-08,"1,241,303.58",RV,NGN,2022-03-08,2022-04-29,1122000225,NGN,1,6022004248,Debrox Contracting Nigeria Limited,52,1241303.58,True,0


Model preparation, train, prediction and evaluation. The model to train here is Logistic Regression

In [26]:
#Using Logistic Regression model

# Select only numerical features for the model
features = ['Invoice Amount', 'Days to Pay']  # Add other relevant numerical features if available
X = data[features]
y = data['Is_Late_Payer']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.linear_model import LogisticRegression

# Initialize and train the model
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print evaluation metrics
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# Print confusion matrix and classification report
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 1.00
Precision: 1.00
Recall: 0.99
F1 Score: 1.00
Confusion Matrix:
[[523   0]
 [  2 216]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       523
           1       1.00      0.99      1.00       218

    accuracy                           1.00       741
   macro avg       1.00      1.00      1.00       741
weighted avg       1.00      1.00      1.00       741



Model preparation, train, prediction and evaluation. The model to train here is Random Forest Classifier model

In [28]:
# Using Random Forest Classifier model

# Convert date columns to datetime
date_columns = ['Document Date', 'Net due date', 'Posting Date', 'Payment date']
for col in date_columns:
    data[col] = pd.to_datetime(data[col], errors='coerce')

# Ensure 'Amount in doc. curr.' and 'Eff.exchange rate' columns are string before replacing commas
data['Amount in doc. curr.'] = data['Amount in doc. curr.'].astype(str).str.replace(',', '')
data['Eff.exchange rate'] = data['Eff.exchange rate'].astype(str).str.replace(',', '')

# Convert these columns to float
data['Amount in doc. curr.'] = data['Amount in doc. curr.'].astype(float)
data['Eff.exchange rate'] = data['Eff.exchange rate'].astype(float)

# Feature Engineering
data['Days_to_due'] = (data['Net due date'] - data['Document Date']).dt.days
data['Days_to_payment'] = (data['Payment date'] - data['Document Date']).dt.days
data['Paid_on_time'] = data['Days_to_payment'] <= data['Days_to_due']

# Encode categorical variables
label_encoder = LabelEncoder()
data['Document type'] = label_encoder.fit_transform(data['Document type'])
data['Local Currency'] = label_encoder.fit_transform(data['Local Currency'])
data['Document currency'] = label_encoder.fit_transform(data['Document currency'])
data['Name 1'] = label_encoder.fit_transform(data['Name 1'])

# Select features and target
# Exclude 'Posting Date' as it is a datetime object
X = data[['Customer_ID', 'Amount in doc. curr.', 'Document type', 'Local Currency', 'Days_to_due']]
y = data['Paid_on_time']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[360  99]
 [ 98 184]]
              precision    recall  f1-score   support

       False       0.79      0.78      0.79       459
        True       0.65      0.65      0.65       282

    accuracy                           0.73       741
   macro avg       0.72      0.72      0.72       741
weighted avg       0.73      0.73      0.73       741

